In [1]:
!pip install datasets


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
TOOL_DEFINITIONS = {
  'send_report_to': 'send_report_to(required String recipient_email):\nSend a report generated based on observations from previous conversations. This can be sent to a health or wellbeing professional trusted by the user. This function should only be called upon the user’s request or with their explicit approval.',
  'make_phone_call': 'make_phone_call(required String phone_number):\nInitiate a phone call to the specified phone number. This function should only be called upon the user’s request or with their explicit approval.',
  'search_memory': 'search_memory(required String query):\nSearch your memory (Waico’s memory, not the user’s) from past conversations for relevant information.\nThe query parameter is a relevant fact, expression, event, person, expression.',
  'get_health_data': 'get_health_data(required String health_data_type, required String period):\nRetrieve health-related data for a specified category and time period.\nThe health_data_type parameter must be one of the following: SLEEP, WATER, STEPS_COUNT, ACTIVE_ENERGY_BURNED, or WEIGHT.\nThe period parameter must be one of: TODAY (from midnight to now) or LAST_24_HOURS.',
  'display_user_progress': 'display_user_progress(required String health_data_type, required String period):\nDisplay a Line Chart of the user’s daily data for a given type and time period. It also shows the total for the given period.\nThe health_data_type parameter must be one of: SLEEP, WATER, STEPS_COUNT, ACTIVE_ENERGY_BURNED, or WEIGHT.\nThe period parameter must be one of: LAST_7_DAYS or LAST_30_DAYS.',
  'create_calendar_single_event': 'create_calendar_single_event(required String event_name, required String starts_at, required String ends_at, optional String description, optional String location):\nCreate a single (non-recurring) calendar event.\nThe starts_at and ends_at parameters should be in ISO 8601 format (e.g. 2024-08-01T10:00:00Z).',
  'schedule_recurring_calendar_event': 'schedule_recurring_calendar_event(required String event_name, required String starts_at, required String ends_at, required String frequency, optional String recurrence_ends_at, optional String description, optional String location):\nSchedule a recurring calendar event.\nThe starts_at and ends_at parameters should be in ISO 8601 format (e.g. 2024-08-01T10:00:00Z).\nThe frequency parameter must be one of the following: DAILY, WEEKLY, MONTHLY, or YEARLY.\nThe recurrence_ends_at parameter is optional and specifies the end date of the recurrence in ISO 8601 format (e.g. 2024-08-01T10:00:00Z). If not provided, the event will recur indefinitely.',
}

In [3]:
import re
import random
from typing import List, Any


def add_system_prompt(sample, lang: str):
    tool_instructions = (
        "\n\nTOOL CALLING:\n"
        "You have access to the tools below that you can call to satisfy a user query or when you think it is appropriate.\n"
        "If a tool requires an input that you don't know or the user query is ambiguous, ask for clarification. If you can't perform the action requested and there is no tool to perform it, politely let the user that you don't have that ability.\n\n"
        f"{"\n---\n".join(get_tool_definitions(sample))}"
    )
    if sample.get("system"):
        sample["system"] += tool_instructions
    else:
        sample["system"] = f"You are Waico, a helpful Wellbeing AI Assistant.{tool_instructions}"
    
    sample["lang"] = lang
    return sample


def get_tool_definitions(sample):
    assistant_message = sample["assistant"]
    tool_match = re.search(r"<tool_call\s+([^\s>]+)>", assistant_message)

    if tool_match:
        tool_called = tool_match.group(1)
        selected_tool_definition = TOOL_DEFINITIONS[tool_called]
        # Choose random tool definitions to include with the tool the model must pick
        # If we provide only the selected tool, the model won't learn to pick the right
        # tool among many available.
        additional_tool_definitions = pick_random_samples(
            data=list(TOOL_DEFINITIONS.values()),
            exclude=selected_tool_definition,
            max_samples=3,
        )
        return [selected_tool_definition, *additional_tool_definitions]
    else:
        # This samples doesn't contain tool call, maybe a case where the user didn't provide
        # enough context for the model to deduce the parameters or there is no tool to
        # perform the action the user requested. In some of these cases, the model will
        # say which tools it can use, to make sure the tools it list are in the system prompt
        # we return all of their definitions here.
        return list(TOOL_DEFINITIONS.values())


def pick_random_samples(
    data: List[Any], max_samples: int, exclude: Any | None = None
) -> List[Any]:
    """
    Randomly picks between 1 and `max_elements` samples from the given list `data`,
    excluding the specified `exclude` element.

    Parameters:
        data (List[Any]): The list to sample from.
        exclude (Any): An element to exclude from sampling.
        max_samples (int): The maximum number of elements to sample (inclusive).

    Returns:
        List[Any]: A list of randomly selected elements (length between 1 and `max_elements`),
                   not including the excluded element.

    Raises:
        ValueError: If no elements remain after exclusion or if max_elements is invalid.
    """
    if exclude is not None:
        data = [item for item in data if item != exclude]

    if not data:
        raise ValueError("No elements left to sample after exclusion.")

    if max_samples < 1:
        raise ValueError("max_samples must be at least 1.")

    max_possible = min(max_samples, len(data))
    sample_size = random.randint(1, max_possible)

    return random.sample(data, sample_size)

In [4]:
from datasets import load_dataset, concatenate_datasets

en_dataset = load_dataset("json", data_files="../datasets/raw/waico-tool-calling-dataset-en.jsonl")["train"]
fr_dataset = load_dataset("json", data_files="../datasets/raw/waico-tool-calling-dataset-fr.jsonl")["train"]
de_dataset = load_dataset("json", data_files="../datasets/raw/waico-tool-calling-dataset-de.jsonl")["train"]
es_dataset = load_dataset("json", data_files="../datasets/raw/waico-tool-calling-dataset-es.jsonl")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
en_dataset = en_dataset.map(add_system_prompt, num_proc=4, fn_kwargs={"lang": "en"})
fr_dataset = fr_dataset.map(add_system_prompt, num_proc=4, fn_kwargs={"lang": "fr"})
de_dataset = de_dataset.map(add_system_prompt, num_proc=4, fn_kwargs={"lang": "de"})
es_dataset = es_dataset.map(add_system_prompt, num_proc=4, fn_kwargs={"lang": "es"})

Map (num_proc=4):   0%|          | 0/1080 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/542 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/316 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/337 [00:00<?, ? examples/s]

In [6]:
prepared_dataset = concatenate_datasets([en_dataset, fr_dataset, de_dataset, es_dataset])

In [7]:
prepared_dataset

Dataset({
    features: ['assistant', 'user', 'system', 'tool', 'assistant_2', 'user_2', 'lang', 'user_3', 'assistant_3'],
    num_rows: 2275
})

In [8]:
prepared_dataset[0]

{'assistant': 'Sure, one moment while I look it up.\n```tool_code\nget_health_data(health_data_type="SLEEP", period="TODAY")\n```',
 'user': 'Can you tell me how many hours I slept Today?',
 'system': "You are Waico, a helpful Wellbeing AI Assistant.\n\nTOOL CALLING:\nYou have access to the tools below that you can call to satisfy a user query or when you think it is appropriate.\nIf a tool requires an input that you don't know or the user query is ambiguous, ask for clarification. If you can't perform the action requested and there is no tool to perform it, politely let the user that you don't have that ability.\n\nsend_report_to(required String recipient_email):\nSend a report generated based on observations from previous conversations. This can be sent to a health or wellbeing professional trusted by the user. This function should only be called upon the user’s request or with their explicit approval.\n---\nmake_phone_call(required String phone_number):\nInitiate a phone call to the

In [9]:
prepared_dataset[900]

{'assistant': 'Sure thing. I\'ve scheduled your shopping trip for this Saturday from 10 AM to 12 PM.\n```tool_code\ncreate_calendar_single_event(event_name="Holiday Shopping", starts_at="2024-12-07T10:00:00Z", ends_at="2024-12-07T12:00:00Z")\n```',
 'user': "Please block out 2 hours for 'Holiday Shopping' this Saturday morning, starting at 10.",
 'system': "You are Waico, a helpful Wellbeing AI Assistant.\nThe current date and time is Thursday, December 5, 2024 – 09:00.\n\nTOOL CALLING:\nYou have access to the tools below that you can call to satisfy a user query or when you think it is appropriate.\nIf a tool requires an input that you don't know or the user query is ambiguous, ask for clarification. If you can't perform the action requested and there is no tool to perform it, politely let the user that you don't have that ability.\n\nsend_report_to(required String recipient_email):\nSend a report generated based on observations from previous conversations. This can be sent to a healt

In [10]:
final_dataset = prepared_dataset.shuffle(seed=11).train_test_split(test_size=275)

In [11]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['assistant', 'user', 'system', 'tool', 'assistant_2', 'user_2', 'lang', 'user_3', 'assistant_3'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['assistant', 'user', 'system', 'tool', 'assistant_2', 'user_2', 'lang', 'user_3', 'assistant_3'],
        num_rows: 275
    })
})

In [12]:
final_dataset.push_to_hub("sitatech/waico-tool-use")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/681 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sitatech/waico-tool-use/commit/b8d6601f1ff8266c2ac1fc462001aa8c1dd2fb6c', commit_message='Upload dataset', commit_description='', oid='b8d6601f1ff8266c2ac1fc462001aa8c1dd2fb6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/sitatech/waico-tool-use', endpoint='https://huggingface.co', repo_type='dataset', repo_id='sitatech/waico-tool-use'), pr_revision=None, pr_num=None)

In [12]:
print("\n---\n".join(TOOL_DEFINITIONS.values()))

send_report_to(required String recipient_email):
Send a report generated based on observations from previous conversations. This can be sent to a health or wellbeing professional trusted by the user. This function should only be called upon the user’s request or with their explicit approval.
---
make_phone_call(required String phone_number):
Initiate a phone call to the specified phone number. This function should only be called upon the user’s request or with their explicit approval. 
---
search_memory(required String query):
Search your memory (Waico’s memory, not the user’s) from past conversations for relevant information.
The query parameter is a relevant fact, expression, event, person, expression.
---
get_health_data(required String health_data_type, required String period):
Retrieve health-related data for a specified category and time period.
The health_data_type parameter must be one of the following: SLEEP, WATER, STEPS_COUNT, ACTIVE_ENERGY_BURNED, or WEIGHT.
The period para